### Name: Yara Hossam El-Din

### Sec: 2

### BN: 48

## Data Preprocessing

In [1]:
#importing libraries
import numpy as np
import pandas as pd

In [2]:
df_ratings=pd.read_csv('ratings.csv')
df_movies=pd.read_csv('movies.csv')

In [3]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
#we only need first 200 users
df_ratings=df_ratings[df_ratings['userId']<=200]

In [5]:
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
29263,200,60074,3.5,1229887390
29264,200,61024,4.0,1229889835
29265,200,61323,4.0,1229887427
29266,200,62299,3.5,1229876803


In [6]:
df_ratings.shape

(29268, 4)

In [7]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [8]:
#we only need first 200 movies
df_movies=df_movies[0:200]

In [9]:
df_movies.shape

(200, 3)

In [10]:
#to get users ids and movies ids in one df, we use merge
#we also drop unnecessary info
df_combined=pd.merge(df_ratings,df_movies,on='movieId')
df_combined.drop(['title','genres','timestamp'],inplace=True,axis=1)
df_combined

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
...,...,...,...
2137,191,148,5.0
2138,191,155,4.0
2139,191,178,1.0
2140,191,229,5.0


In [11]:
#to get the unique movies among the first 200 movies that all our 200 users watched
df_combined['movieId'].nunique()

176

In [12]:
#to get a rating table of user vs movie (this is what we pass to cosine similarity)
rating_pivot = df_combined.pivot_table(index=['movieId'],columns=['userId'], values=['rating']).fillna(0)
rating_pivot

rating                                               ...            \
userId     1    3    4    5    6    7    8    9    11   12   ...  191  192   
movieId                                                      ...             
1          4.0  0.0  0.0  4.0  0.0  4.5  0.0  0.0  0.0  0.0  ...  4.0  0.0   
2          0.0  0.0  0.0  0.0  4.0  0.0  4.0  0.0  0.0  0.0  ...  0.0  0.0   
3          4.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4          0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
5          0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
229        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  5.0  0.0   
230        0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  ...  4.0  0.0   
231        5.0  0.0  0.0  0.0  3.0  0.0  4.0  0.0  0.0  0.0  ...  0.0  0.0   
232        0.0  0.0  5.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
233        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

                                                 
userId   193  194  195  196  197  198  199  200  
movieId                                          
1        2.0  0.0  0.0  0.0  0.0  0.0  0.0  3.5  
2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5        0.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  
...      ...  ...  ...  ...  ...  ...  ...  ...  
229      0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  
230      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
231      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
232      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
233      0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  

[176 rows x 180 columns]

In [13]:
rating_pivot.index

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            223, 224, 225, 227, 228, 229, 230, 231, 232, 233],
           dtype='int64', name='movieId', length=176)

## Implimenting a recommender system

In [14]:
#we calc cosine similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix=cosine_similarity(rating_pivot)

In [15]:
sim_matrix.shape

(176, 176)

In [16]:
#we convert sim matrix into a df for more convenience
df_sim = pd.DataFrame(sim_matrix)
df_sim

,0,1,2,3,4,5,6,7,8,9,...,166,167,168,169,170,171,172,173,174,175
0,1.000000,0.364634,0.284825,0.000000,0.279775,0.327928,0.270701,0.128765,0.195017,0.352899,...,0.363093,0.211791,0.160623,0.088024,0.107733,0.104516,0.140704,0.360625,0.087884,0.000000
1,0.364634,1.000000,0.278446,0.102937,0.277072,0.278936,0.240488,0.130496,0.000000,0.342590,...,0.202565,0.085855,0.132766,0.114888,0.000000,0.000000,0.191316,0.352531,0.000000,0.000000
2,0.284825,0.278446,1.000000,0.150278,0.391915,0.341700,0.592157,0.406427,0.288534,0.231410,...,0.284289,0.200545,0.347200,0.197325,0.000000,0.000000,0.244260,0.485324,0.000000,0.000000
3,0.000000,0.102937,0.150278,1.000000,0.239259,0.176604,0.297614,0.253546,0.000000,0.129040,...,0.108721,0.382276,0.280386,0.000000,0.000000,0.000000,0.369406,0.132357,0.257513,0.000000
4,0.279775,0.277072,0.391915,0.239259,1.000000,0.256166,0.539392,0.404422,0.000000,0.275293,...,0.150873,0.295177,0.380707,0.070687,0.000000,0.000000,0.287247,0.296883,0.092419,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.104516,0.000000,0.000000,0.000000,0.000000,0.242299,0.091873,0.000000,0.000000,0.000000,...,0.253580,0.238400,0.000000,0.000000,0.000000,1.000000,0.316763,0.000000,0.000000,0.000000
172,0.140704,0.191316,0.244260,0.369406,0.287247,0.305806,0.346312,0.234154,0.000000,0.154921,...,0.325314,0.481415,0.258941,0.109137,0.326512,0.316763,1.000000,0.142605,0.256842,0.142543
173,0.360625,0.352531,0.485324,0.132357,0.296883,0.295106,0.357803,0.178979,0.079414,0.411800,...,0.335287,0.142591,0.487390,0.301241,0.187180,0.000000,0.142605,1.000000,0.000000,0.068097
174,0.087884,0.000000,0.000000,0.257513,0.092419,0.090956,0.116492,0.000000,0.000000,0.066459,...,0.183661,0.143187,0.086644,0.047338,0.000000,0.000000,0.256842,0.000000,1.000000,0.000000


In [17]:
df_sim.index
# notice index of each movie in this df is different from the movie id
#so we need to map each idx to id

RangeIndex(start=0, stop=176, step=1)

In [18]:
# we create a dic to map movie indicies with their ids
idx_to_id=dict(zip(df_sim.index,rating_pivot.index))

In [19]:
def get_movie_idx_from_id(ID):
    """a function to return movie index given the movie id """
    for key, value in idx_to_id.items():
         if ID == value:
            return key

In [20]:
def recommend_based_on_movie(movie_title,df_movies):
    """ a function to return a list of most 10 similar movies given a movie title"""
    movie_id = int(df_movies['movieId'][df_movies['title']==movie_title])
    movie_idx = get_movie_idx_from_id(movie_id)
    movies_list = []
    movies_ids = []
    #we get the indicies of movies with highest 10 similarity scores with this movie
    sim_scores = df_sim[movie_idx].sort_values(ascending = False)[1:11]
    sim_scores_df = sim_scores.to_frame()
    #now we get the movie ids of these movies
    idx_to_id = dict(zip(df_sim.index,rating_pivot.index))
    mov_index = sim_scores_df.index
    for i in mov_index:
        movies_ids.append(idx_to_id[i])
    #we get titles of the recommended movies          
    for mov_id in movies_ids:
        rec_movie = df_movies['title'][df_movies['movieId']==mov_id]
        movies_list.append(rec_movie)
    return movies_list

## Testing our recommender system

In [21]:
movie1_recs=recommend_based_on_movie( "Toy Story (1995)",df_movies)
movie1_recs

[97    Braveheart (1995)
 Name: title, dtype: object,
 123    Apollo 13 (1995)
 Name: title, dtype: object,
 46    Usual Suspects, The (1995)
 Name: title, dtype: object,
 92    Happy Gilmore (1996)
 Name: title, dtype: object,
 138    Die Hard: With a Vengeance (1995)
 Name: title, dtype: object,
 43    Seven (a.k.a. Se7en) (1995)
 Name: title, dtype: object,
 32    Babe (1995)
 Name: title, dtype: object,
 176    Waterworld (1995)
 Name: title, dtype: object,
 126    Batman Forever (1995)
 Name: title, dtype: object,
 98    Taxi Driver (1976)
 Name: title, dtype: object]

In [22]:
movie2_recs=recommend_based_on_movie( "Waiting to Exhale (1995)",df_movies)
movie2_recs

[100    Before and After (1996)
 Name: title, dtype: object,
 153    Mighty Morphin Power Rangers: The Movie (1995)
 Name: title, dtype: object,
 182    Before the Rain (Pred dozhdot) (1994)
 Name: title, dtype: object,
 160    Reckless (1995)
 Name: title, dtype: object,
 187    Cure, The (1995)
 Name: title, dtype: object,
 169    Three Wishes (1995)
 Name: title, dtype: object,
 50    Georgia (1995)
 Name: title, dtype: object,
 151    Mad Love (1995)
 Name: title, dtype: object,
 177    White Man's Burden (1995)
 Name: title, dtype: object,
 180    Bushwhacked (1995)
 Name: title, dtype: object]